### Passo 1 - Importar Arquivos e Bibliotecas

In [ ]:
import pandas as pd
import yagmail
from pathlib import Path

In [111]:
df_vendas = pd.read_excel(r'Bases de Dados/Vendas.xlsx')
df_emails = pd.read_excel(r'Bases de Dados/Emails.xlsx')
df_lojas = pd.read_csv(r'Bases de Dados/Lojas.csv', encoding='latin1', sep=';')

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [65]:
df_vendas_e_loja = df_vendas.merge(df_lojas, on='ID Loja')

dic_vendas = {}

for loja in df_lojas['Loja']:
    dic_vendas[loja] = df_vendas_e_loja.loc[df_vendas_e_loja['Loja'] == loja, :]

# definir o dia do indicador
indicador_dia = df_vendas['Data'].max()
indicador_dia.day

### Salvar a planilha na pasta de backup

In [6]:
caminho = Path('Backup Arquivos Lojas')
arquivo_pasta_backup = caminho.iterdir()


lista_nomes_backup = [arquivo.name for arquivo in arquivo_pasta_backup]

for nome in dic_vendas.keys():
    if nome not in lista_nomes_backup:
        Path(f'{caminho}/{nome}').mkdir()

    dic_vendas[nome].to_excel(f'{caminho}/{nome}/{indicador_dia.month}_{indicador_dia.day}_{nome}.xlsx', index=False)

###  Enviar por e-mail para o gerente

In [9]:
usuario = yagmail.SMTP(user='*******@gmail.com', password='******')

gerentes_df = pd.read_excel('Bases de Dados/Emails.xlsx')


###  Automatizar todas as lojas

In [38]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtde_dia = 4
meta_qtde_ano = 120
meta_ticket_dia = 500
meta_ticket_ano = 500

def cenario(valor, meta):
    if valor > meta:
        cor = 'green'
    else:
        cor = 'red'
    return cor 

In [63]:
for i, loja in enumerate(dic_vendas):
    vendas_loja = dic_vendas[loja]
    filtro_data = vendas_loja.loc[vendas_loja['Data']== indicador_dia, :]

    # faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = filtro_data['Valor Final'].sum()

    #diverdidade produto
    diversidade_prod_ano = len(vendas_loja['Produto'].unique())
    diversidade_prod_dia = len(filtro_data['Produto'].unique())

    #ticket medio
    valor_agrupado_ano = vendas_loja.groupby(vendas_loja['Código Venda'])['Valor Final'].sum()
    ticket_medio_ano = valor_agrupado_ano.mean()
    valor_agrupado_dia = filtro_data['Valor Final'].groupby(vendas_loja['Código Venda']).sum()
    ticket_medio_dia = valor_agrupado_dia.mean()

    #mandar email
    assunto_email = f'OnePage Dia {indicador_dia.day}/{indicador_dia.month}/{indicador_dia.year} - Loja {loja}'
    corpo_email = f'''
    Bom dia {gerentes_df["Gerente"][i]},

    O resultado de ontem ({indicador_dia.day}/{indicador_dia.month}) da Loja {loja} foi de:

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
    </tr>
    <tr>
        <td style="text-align:center">Faturamento</td>
        <td style="text-align:center">{faturamento_dia}</td>
        <td style="text-align:center">{meta_faturamento_dia}</td>
        <td style="text-align:center"><font color="{cenario(faturamento_dia, meta_faturamento_dia)}">◙</font></td>
        <td style="text-align:center">{faturamento_ano}</td>
        <td style="text-align:center">{meta_faturamento_ano}</td>
        <td style="text-align:center"><font color="{cenario(faturamento_ano, meta_faturamento_ano)}">◙</font></td>
    </tr>
    <tr>
        <td style="text-align:center">Diversidade dos produtos</td>
        <td style="text-align:center">{diversidade_prod_dia}</td>
        <td style="text-align:center">{meta_qtde_dia}</td>
        <td style="text-align:center"><font color="{cenario(diversidade_prod_dia, meta_qtde_dia)}">◙</font></td>
        <td style="text-align:center">{diversidade_prod_ano}</td>
        <td style="text-align:center">{meta_qtde_ano}</td>
        <td style="text-align:center"><font color="{cenario(diversidade_prod_ano, meta_qtde_dia)}">◙</font></td>
    </tr>
    <tr>
        <td style="text-align:center">Ticket Médio</td>
        <td style="text-align:center">{ticket_medio_dia}</td>
        <td style="text-align:center">{meta_ticket_dia}</td>
        <td style="text-align:center"><font color="{cenario(ticket_medio_dia, meta_ticket_dia)}">◙</font></td>
        <td style="text-align:center">{ticket_medio_ano}</td>
        <td style="text-align:center">{meta_ticket_ano}</td>
        <td style="text-align:center"><font color="{cenario(ticket_medio_ano, meta_ticket_ano)}">◙</font></td>
    </tr>
    </table>

    Segue anexo a planilha com todos os dados para mais detalhes. Qualquer dúvida estou à disposição.

    Att.,
    Fulano 
        
    '''
    anexo = Path.cwd()/caminho/loja/f'{indicador_dia.month}_{indicador_dia.day}_{loja}.xlsx'
    usuario.send(to=gerentes_df["E-mail"][i], subject= assunto_email, contents= corpo_email, attachments=anexo)

    print(f'email {loja} enviado')
        

email Iguatemi Esplanada enviado
email Shopping Midway Mall enviado
email Norte Shopping enviado
email Shopping Iguatemi Fortaleza enviado
email Shopping Uni�o de Osasco enviado
email Shopping Center Interlagos enviado
email Rio Mar Recife enviado
email Salvador Shopping enviado
email Rio Mar Shopping Fortaleza enviado
email Shopping Center Leste Aricanduva enviado
email Ribeir�o Shopping enviado
email Shopping Morumbi enviado
email Parque Dom Pedro Shopping enviado
email Bourbon Shopping SP enviado
email Palladium Shopping Curitiba enviado
email Passei das �guas Shopping enviado
email Center Shopping Uberl�ndia enviado
email Shopping Recife enviado
email Shopping Vila Velha enviado
email Shopping SP Market enviado
email Shopping Eldorado enviado
email Shopping Ibirapuera enviado
email Novo Shopping Ribeir�o Preto enviado
email Iguatemi Campinas enviado
email Shopping Barra enviado


### Criar ranking para diretoria

In [89]:
faturamento_lojas_ano = df_vendas_e_loja.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_ano)

vendas_dia = df_vendas_e_loja.loc[df_vendas_e_loja['Data'] == indicador_dia, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

faturamento_lojas_ano.to_excel(f'{caminho}/{indicador_dia.month}_{indicador_dia.day}_Ranking Anual.xlsx')
faturamento_lojas_dia.to_excel(f'{caminho}/{indicador_dia.month}_{indicador_dia.day}_Ranking Dia.xlsx')


,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeir�o Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeir�o Preto,3400
Center Shopping Uberl�ndia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping Uni�o de Osasco,1207
Shopping Vila Velha,937


### Enviar e-mail para diretoria

In [108]:
diretoria = gerentes_df.loc[gerentes_df['Loja'] == 'Diretoria', 'E-mail'].values[0]
assunto_email = f' Ranking Dia {indicador_dia.day}/{indicador_dia.month}'
corpo_email = f'''
Prezados, bom dia!

Melhor loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com faturamento de R$ {faturamento_lojas_dia.iloc[0,0]:.2f}
Pior loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com faturamento de R$ {faturamento_lojas_dia.iloc[-1,0]:.2f}

Melhor loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com faturamento de R$ {faturamento_lojas_ano.iloc[0,0]:.2f}
Pior loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[-1]} com faturamento de R$ {faturamento_lojas_ano.iloc[-1,0]:.2f}

Segue anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida fico à disposição.

Att.
'''
ranking_dia = Path.cwd()/caminho/f'{indicador_dia.month}_{indicador_dia.day}_Ranking Dia.xlsx'
ranking_ano = Path.cwd()/caminho/f'{indicador_dia.month}_{indicador_dia.day}_Ranking Anual.xlsx'
usuario.send(to=diretoria, subject= assunto_email, contents= corpo_email, attachments=[ranking_dia, ranking_ano])

print(f'email Diretoria enviado')

email Diretoria enviado
